In [ ]:
from pathlib import Path
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import pandas as pd
import json
import tqdm
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset, DatasetDict
import evaluate
import os

os.environ["CUDA_VISIIBLE_DEVICES"] = "0,1"
NUM_EPOCHS = 50
EXPERIMENT_NAME = "t5-small_falcon2-default_annotation-default"
EXPERIMENT_DIR = Path('experiments')
MODEL_ARTIFACTS = EXPERIMENT_DIR / EXPERIMENT_DIR
WEIGHTS_DIR = MODEL_ARTIFACTS / 'weights'
VALS_DIR = MODEL_ARTIFACTS / 'validations'

Make appropriate directoreis

In [ ]:
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)
VALS_DIR.mkdir(parents=True, exist_ok=True)

Defining the model and tokenizer

In [ ]:
model_path = "t5-small"
tokenizer_path = "t5-small"

model = T5ForConditionalGeneration.from_pretrained(model_path).to("cuda")
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path).to("cuda")

Define dataset maker

In [ ]:
def split_dataframe(df):
  # ratios from Bannerjee
  train = 0.7
  dev = 0.1
  test = 0.2
  assert train + dev + test == 1.0
  data_len = len(df)
  train_set = Dataset.from_pandas(df[:round(data_len * train)])
  dev_set = Dataset.from_pandas(df[round(data_len * train):round(data_len* (train + dev))])
  test_set = Dataset.from_pandas(df[round(data_len * (train + dev)):])
  
  dataset = DatasetDict()
  dataset['train'] = train_set
  dataset['dev'] = dev_set
  dataset['test'] = test_set

  return dataset

Define dataset tokenizer

In [ ]:
def tokenize_data(dataset, column):
  model_inputs = tokenizer(dataset[column], padding=True, return_tensors="pt")
  return model_inputs

Defining the validation function

In [ ]:
def val(val_dataloader, val_path = None):
  model.eval()
  eval_dict = []
  for val_batch in val_dataloader:
    batch = {}
    for k,v in val_batch.items():
      if k in set("input_ids", "labels", "attention_mask"):
        batch[k] = v.to("cuda")
    
    with torch.no_grad():
      outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    for i, pred in enumerate(tokenizer.batch_decode(predictions)):
      eval_dict.append({
        "Utte": val_batch['utterance'][i],
        "Anno": val_batch['annotated'][i],
        "Gold": val_batch['gold'][i],
        "Gene": pred, # THIS NEEDS TO BE UNMASKED
      })
  
  if val_path:
    with open(val_path, "w") as f:
      json.dump(eval_dict, f, indent=2)
  
  model.train()
  return eval_dict

In [ ]:
def training_loop(df):

  assert 'utterance' in df.columns
  assert 'annotated' in df.columns
  assert 'gold' in df.columns

  dataset = split_dataframe(df)
  tokenized_dataset = dataset \
    .map(lambda x: tokenize_data(x, 'gold'), batched=True) \
    .rename_column('input_ids', 'labels') \
    .map(lambda x: tokenize_data(x, 'annotated'), batched=True)
  
  train_dataset = tokenize_data["train"]
  dev_dataset = tokenize_data["dev"]
  test_dataset = tokenize_data["test"]

  train_dataloader = DataLoader(train_dataset, batch_size = 2)
  dev_dataloader = DataLoader(dev_dataset, batch_size = 2)

  scalar = 0
  i = 0

  optimizer = optim.AdamW(model.parameters(), lr = 0.0015)
  lr_scheduler=transformers. \
    get_polynomial_decay_schedule_with_warmup(optimizer, 5000, 30000, power=0.5)
  for epoch in range(num_epochs):
    i = 0
    iters = len(train_dataloader)
    for batch in train_dataloader:
      newbatch = {}
      for k,v in batch.items():
        if k not in ["label", "input"]:
          newbatch[k] = v.to("cuda")
      
      batch = newbatch

      outputs = model(**batch)
      loss = outputs.loss
      scalar += loss.mean().item()

      if (i + 1) % 100 == 0:
        print(f'iteration = {i + 1}/{iters}, training loss={scalar/100}')
        scalar = 0

      loss /= 10 
      loss.mean().backward()
      if (i+1) % 10:
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
      
      i += 1
    
    val(val_dataloader, VALS_DIR / f"val_{epoch}.json")

    torch.save(model.state_dict(),
      WEIGHTS_DIR / f"cp_{epoch}.pth")


